In [2]:
import numpy as np
import re
from collections import Counter
import random
from typing import List, Tuple, Dict
from textwrap import dedent
from dataclasses import dataclass
from tqdm.notebook import tqdm, trange

In [3]:
CORPUS1_RUS_PATH = "data/WarAndPeace.txt"
CORPUS2_RUS_PATH = "data/AnnaKarenina.txt"

In [4]:
with open(CORPUS1_RUS_PATH, mode="r") as f1, open(CORPUS2_RUS_PATH, mode="r") as f2:
    part1 = f1.read()
    part2 = f2.read()
    corpus_rus = part1 + " " + part2

In [5]:
def apply_mapping_unigrams(text: str, mapping: Dict[str, str]) -> str:
    return "".join(mapping.get(s, s) for s in text)

In [6]:
class Text:
    def __init__(self, text: str) -> None:
        text = self.preprocess(text)
        self.decoded = text
        self.encoded = self.encode(text)
    
    @staticmethod
    def encode(text: str) -> Tuple[str, Dict[str, str], Dict[str, str]]:
        chars = list(set(text))
        shuffled_chars = random.sample(chars, k=len(chars))
        encode_mapping = dict(zip(chars, shuffled_chars))
        encoded_text = apply_mapping_unigrams(text, encode_mapping)
        return encoded_text
    
    @staticmethod
    def preprocess(text: str) -> str:
        text = re.sub("[^а-яА-ЯёЁ\s]", " ", text)
        text = text.replace("ё", "е").replace("Ё", "е")
        text = " ".join(text.split())
        return text.lower()

In [7]:
def sort_chars_by_frequency_unigrams(text: str) -> List[str]:
    return [s for s, _ in Counter(text).most_common()]

def hamming_accuracy(text1: str, text2: str) -> float:
    assert len(text1) == len(text2)
    numerator = 0
    for i in range(len(text1)):
        if text1[i] == text2[i]:
            numerator += 1
    denominator = len(text1) or 1
    return numerator / denominator

<b>
1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
расшифруйте их таким частотным методом.
</b>

In [8]:
def simple_frequency_unigrams_translate(train_text: str, test_text: str) -> str:
    train_text = Text.preprocess(train_text)
    train_dictionary = sort_chars_by_frequency_unigrams(train_text)
    
    test_text = Text(test_text)
    encoded_test_text = test_text.encoded
    test_dictionary = sort_chars_by_frequency_unigrams(encoded_test_text)

    mapping = dict(zip(test_dictionary, train_dictionary))
    res = apply_mapping_unigrams(encoded_test_text, mapping)
    score = hamming_accuracy(res, test_text.decoded)
    print(f"Hamming Accuracy: {score}")
    return res

In [9]:
corpus_rus = Text.preprocess(corpus_rus)

assert corpus_rus == simple_frequency_unigrams_translate(corpus_rus, corpus_rus)

Hamming Accuracy: 1.0


In [10]:
test_text = dedent("""
    В автобусе было душно и очень тесно. Старика зажали со всех сторон, и он уже сто раз пожалел о том, что 
    решил поехать на очередной прием к врачу ранним утром. Он ехал и думал о том, что совсем, казалось бы, 
    недавно, а на самом деле семьдесят лет тому назад, он ездил на автобусе в школу. А потом началась война. Он 
    не любил вспоминать то, что он пережил там. Больно было вспоминать тех, кто ушёл с ним добровольцем на фронт 
    и не вернулся. Война была для него и личной трагедией: во время боёв под Москвой и Сталинградом погибли его 
    отец и старший брат. Но был в его жизни один случай, который он тоже не мог забыть и простить себе. 
    Он ехал на автобусе в школу (он тогда учился в третьем классе), сел на последнее свободное место, 
    отвернувшись от старика, который беспомощно облокотился о поручень у дверей. Он не заметил, где сошёл старик,
    но почему-то весь день потом вспоминал о нём, и острая поздняя боль раскаяния пронзила его душу. «Почему я 
    не уступил ему место?» – этот вопрос терзал день изо дня. Потом постепенно это воспоминание ушло на задний 
    план, но время от времени возвращалось как укол совести, как руководство к правильному поведению, к уважению 
    старших и поклону их опыту и седине. Теперь, когда он сам стал таким же немощным стариком, ему было до слёз 
    обидно, если он сталкивался с неуважительным отношением к людям пожилого возраста, к ветеранам. Автобус 
    остановился на остановке, пассажиры начали выходить, стоять стало свободнее. Вдруг к нему подошёл мальчик лет
    десяти и сказал: «Садитесь, дедушка, на моё место, мне кажется, Вам тяжело стоять». У старика навернулись 
    слёзы на глаза. Это были одновременно и горькие, и сладкие слёзы. Они горчили потому, что совесть опять 
    напомнила случай семидесятилетней давности, но они радовали и согревали сердце потому, что он благодаря этому
    мальчику верил, что не всё потеряно для русского человека. 
""")

In [11]:
simple_frequency_unigrams_translate(corpus_rus, test_text)

Hamming Accuracy: 0.6233333333333333


'в авиогмте гчло дмжно с оыеня иетно тиаксуа башалс то втех тиокон с он мше тио каб пошалел о иор ыио кежсл поехаия на оыекедной пксер у вкаым каннср микор он ехал с дмрал о иор ыио товтер уабалотя гч недавно а на тарор деле терядетьи леи иорм набад он ебдсл на авиогмте в жуолм а поиор наыалатя война он не люгсл втпорснаия ио ыио он пекешсл иар голяно гчло втпорснаия иех уио мжел т нср догковоляцер на фкони с не векнмлть война гчла дль незо с лсыной иказедсей во вкерь гоев под ротувой с тиалснзкадор позсглс езо оиец с тиакжсй гкаи но гчл в езо шсбнс одсн тлмыай уоиокчй он иоше не роз багчия с пкотисия теге он ехал на авиогмте в жуолм он иозда мыслть в икеияер улатте тел на потледнее твогодное ретио оивекнмвжстя ои тиаксуа уоиокчй гетпорощно оглоуоислть о покмыеня м двекей он не бареисл зде тожел тиаксу но поыерм ио ветя деня поиор втпорснал о нер с отикаь побдньь голя катуаьнсь пконбсла езо дмжм поыерм ь не мтимпсл ерм ретио эиои вопкот иекбал деня сбо днь поиор потиепенно эио вотпорсн

<i><strong>
Был взят достаточно большой текст, и в целом получилось неплохо по метрике (выше ожиданий), но все-таки нечитаемо.
</strong></i>

<b>
2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
проведите тестирование аналогично п.1, но при помощи биграмм.
</b>

In [12]:
def get_bigrams(text: str) -> List[str]:
    return [text[i:i + 2] for i in range(0, len(text) - 1, 2)]


def sort_chars_by_frequency_bigrams(text: str) -> List[str]:
    bigrams = get_bigrams(text)
    return [s for s, _ in Counter(bigrams).most_common()]

In [13]:
def apply_mapping_bigrams(text: str, mapping: Dict[str, str]) -> str:
    res = "".join([
        mapping.get(text[i:i + 2], text[i:i + 2])
        for i in range(0, len(text) - 1, 2)
    ])
    return res


def simple_frequency_bigrams_translate(train_text: str, test_text: str) -> str:
    train_text = Text.preprocess(train_text)
    train_dictionary = sort_chars_by_frequency_bigrams(train_text)
    
    test_text = Text(test_text)
    encoded_test_text = test_text.encoded
    test_dictionary = sort_chars_by_frequency_bigrams(encoded_test_text)

    mapping = dict(zip(test_dictionary, train_dictionary))
    mapped = apply_mapping_bigrams(encoded_test_text, mapping)
    if len(mapped) != len(test_text.decoded):
        mapped += " "
    score = hamming_accuracy(mapped, test_text.decoded)
    print(f"Hamming Accuracy: {score}")
    return mapped

In [14]:
assert corpus_rus == simple_frequency_bigrams_translate(corpus_rus, corpus_rus)

Hamming Accuracy: 1.0


In [15]:
simple_frequency_bigrams_translate(corpus_rus, test_text)

Hamming Accuracy: 0.08888888888888889


'сь уа мениколие изсае  иннла о миче наорил врадр до е рит о а аз к им варео а бых ал э чим сенонзаа быудкаалмишин и  вннвокинеже гемя  едеужгороикднвамуон с кт  чпоовид ч сенонзаа о инния  брал такоестьэтиве  всто тионовим пниглиттеоломола ски х ек с кмаань сттраматве псьокасго в на я стаяу татодост с ктьслочкатокронку х оа заа  с ктык нуь ноя еты неколие ри негстн ент й а ваодь выотя лесиинасих ти  вжим ол итьтовольтвпрелжа вруу овейи еде  и дятдоенротуанийтое де тпретчиалакылеррыдопоо но дщеролея  нвнсо иеде осыйпоо ноиеаркороолнеколито зоееписот сан квию  жй осседи с ка реи  п лымрарун поалазнаше ониош с кт  чи  в уа менитомноглс с ка вс вю катетоенк н  тй у хо пниь сталеромв чао елетв миылоба  севвольулпе оосо нонь бй осседикооб н лебе атл огдатвпре  надчтлуваовлок жем и  прачедаь вс пмоодь наорили е  нчтскене лотаовла о на я ри негсть е тья  иермуняалажв пакоас оро яняотпр гм ьку  зоеовойго нчтскбри  пвеутвшь  тгоченае боостоавазвы мз  човла оисе в пр на я  ннабеланеееа тоер негс

<i><strong>
Неудивительно, что результат хуже, ведь для "правильного" ранжирования биграмм необходим куда больший тестовый корпус, в виду их большего разнообразия по сравнению с юниграммами.
</strong></i>

<b>
3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
реализуйте и протестируйте его, убедитесь, что результаты улучшились.
</b>

In [16]:
def get_all_bigrams(text: str) -> List[str]:
    return [text[i:i + 2] for i in range(len(text) - 2)]

In [17]:
def get_freqs(text: str) -> Dict[str, float]:
    bigrams = get_all_bigrams(text)
    freqs = {}
    for k, v in Counter(bigrams).items():
        freqs[k] = v / len(bigrams)
    return freqs

def text_proba(text: str, mapping: Dict[str, str], freqs: Dict[str, float]) -> float:
    decoded_text = apply_mapping_unigrams(text, mapping)
    bigrams = get_all_bigrams(decoded_text)
    log_proba = 0
    for b in bigrams:
        log_proba += np.log(freqs.get(b, 1 / len(bigrams)))
    return log_proba

In [18]:
@dataclass
class MCMC:
    train_text: str
    test_text: str
    num_iterations: int = 10000
    num_epochs: int = 10
    verbose: bool = False
    
    def __post_init__(self):
        self.best_mapping = None
        self.best_log_proba = -np.inf
        self.train_alphabet = list(set(self.train_text))
        self.test_alphabet = list(set(self.test_text))
        self.train_freqs = get_freqs(self.train_text)
    
    def fit(self):
        for e in trange(self.num_epochs):
            cur_mapping = self.get_mapping(self.test_alphabet, self.train_alphabet)
            cur_log_proba = text_proba(self.test_text, cur_mapping, self.train_freqs)
            trained_alphabet = self.train_alphabet[:]
            
            for i in tqdm(range(self.num_iterations), leave=False):
                proposal_alphabet = self.transposition(trained_alphabet)
                proposal_mapping = self.get_mapping(self.test_alphabet, proposal_alphabet)
                proposal_log_proba = text_proba(self.test_text, proposal_mapping, self.train_freqs)
                
                p_proba = np.clip(np.exp(proposal_log_proba - cur_log_proba), 0, 1)
                
                if p_proba > random.random():
                    cur_log_proba = proposal_log_proba
                    cur_mapping = proposal_mapping
                    trained_alphabet = proposal_alphabet[:]
                    
            if cur_log_proba > self.best_log_proba:
                self.best_log_proba = cur_log_proba
                self.best_mapping = cur_mapping
            print(f"Epoch: {e}, Log-likelihood: {self.best_log_proba}")
            if self.verbose:
                print()
                self.inference()
                
    def inference(self):
        res = apply_mapping_unigrams(self.test_text, self.best_mapping)
        print(res)
        
    @staticmethod
    def transposition(alphabet):
        res = alphabet[:]
        idx1, idx2 = np.random.choice(len(res), size=2, replace=False)
        res[idx1], res[idx2] = res[idx2], res[idx1]
        return res
    
    @staticmethod
    def get_mapping(alphabet1, alphabet2):
        return dict(zip(alphabet1, alphabet2))

In [19]:
encoded_text = Text(test_text).encoded
print(encoded_text)

жвижуйестзвеомйвясцлйвнвйбзльвузтлйвтуиандивкихимнвтйвжтзрвтуйайлвнвйлвсхзвтуйваиквюйхимзмвйвуй вбуйвазцнмвюйзриуьвливйбзазялйщвюанз вдвжаибсваиллн всуай вйлвзримвнвяс имвйвуй вбуйвтйжтз вдикимйтьвеовлзяижлйвивливти й вязмзвтз ьязтпувмзувуй свликиявйлвзкянмвливижуйестзвжвцдймсвивюйуй влибимитьвжйщливйлвлзвмгенмвжтюй нлиуьвуйвбуйвйлвюзазхнмвуи веймьлйвеомйвжтюй нлиуьвузрвдуйвсцзмвтвлн вяйеайжймьфз вливэайлувнвлзвжзалсмтпвжйщливеомивямпвлзыйвнвмнблйщвуаиызянзщвжйвжаз пвейзжвюйяв йтджйщвнвтуимнлыаияй вюйынемнвзыйвйузфвнвтуиацнщвеаиувлйвеомвжвзыйвхнклнвйянлвтмсбищвдйуйаощвйлвуйхзвлзв йывкиеоуьвнвюайтунуьвтзезвйлвзримвливижуйестзвжвцдймсвйлвуйыяивсбнмтпвжвуазуьз вдмиттзвтзмвливюйтмзялззвтжйейялйзв зтуйвйужзалсжцнтьвйувтуиандивдйуйаощвезтюй йшлйвйемйдйунмтпвйвюйасбзльвсвяжзазщвйлвлзвки зунмвыязвтйцзмвтуиандвлйвюйбз свуйвжзтьвязльвюйуй вжтюй нлимвйвлз внвйтуаипвюйкялппвеймьваитдиплнпвюайлкнмивзыйвясцсвюйбз свпвлзвстусюнмвз св зтуйвчуйувжйюайтвузакимвязльвнкйвялпвюйуй вюйтузюзллйвчуйвжйтюй нли

In [20]:
mcmc = MCMC(corpus_rus, encoded_text, num_epochs=3, num_iterations=10000)

mcmc.fit()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 0, Log-likelihood: -9656.335559151188


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1, Log-likelihood: -9656.335559151188


  0%|          | 0/10000 [00:00<?, ?it/s]

<ipython-input-18-351ff23d354c>:27: RuntimeWarning: overflow encountered in exp
  p_proba = np.clip(np.exp(proposal_log_proba - cur_log_proba), 0, 1)


Epoch: 2, Log-likelihood: -9656.335559151188


In [21]:
mcmc.inference()

в автобусе было душно и очень тесно старика зажали со всех сторон и он уже сто раз пожалел о том что решил поехать на очередной прием к врачу ранним утром он ехал и думал о том что совсем казалось бы недавно а на самом деле семьдесят лет тому назад он ездил на автобусе в школу а потом началась война он не любил вспоминать то что он пережил там больно было вспоминать тех кто ушел с ним добровольцем на фронт и не вернулся война была для него и личной трагедией во время боев под москвой и сталинградом погибли его отец и старший брат но был в его жизни один случай который он тоже не мог забыть и простить себе он ехал на автобусе в школу он тогда учился в третьем классе сел на последнее свободное место отвернувшись от старика который беспомоъно облокотился о поручень у дверей он не заметил где сошел старик но почему то весь день потом вспоминал о нем и острая поздняя боль раскаяния пронзила его душу почему я не уступил ему место этот вопрос терзал день изо дня потом постепенно это воспомина

<b>
4. Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
</b>

In [22]:
encoded_message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [23]:
mcmc = MCMC(corpus_rus, encoded_message, num_epochs=10, num_iterations=1000000, verbose=True)

mcmc.fit()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

Epoch: 0, Log-likelihood: -1307.5529872387574

нъитйжпйжтотанй чщылим прйтитйэчяатй чщылим прйануъайхйьачцчйъччден твйучачщпрйинцучйэщчяталамйъучщннйжънцчйжпйжънйъонилитйэщлжтим чйтйэчихятанйылуътылим прйдлиийглйэчъино ннйянажнщачнйглол тнйухщълйючавйуч ня чйвй тянцчй нйчднелф


  0%|          | 0/1000000 [00:00<?, ?it/s]

Epoch: 1, Log-likelihood: -1299.9407936684152

йьногхзгхосолйгпъшжен пзфгоногцъвлогпъшжен пзфглйыьлгагилъуъгьъъщюйпочгыълъшзфгнйуыъгцшъволел гьыъшййгхьйуъгхзгхьйгьсйненогцшехон пъгогцънаволйгжеыьожен пзфгщеннгэегцъьнйспййгвйлхйшлъйгэесепойгыашьегяълчгыъпйвпъгчгповйуъгпйгъщйюеб


...

Epoch: 7, Log-likelihood: -1210.2653009748797

если чы читине воъзалйвыь или подни воъзалйвыь нексн у жноро соошэевия коноъыь лерко пъодинанй скоъее чсеро чы чсе стелали пъачилйво и полудине заксизалйвыь шалл ма послетвее денчеъное матавие куъса гоня коведво я видеро ве ошеэащ


  0%|          | 0/1000000 [00:00<?, ?it/s]

Epoch: 8, Log-likelihood: -1210.2653009748797

если чы читине воъзалйвыь или подни воъзалйвыь нексн у жноро соошэевия коноъыь лерко пъодинанй скоъее чсеро чы чсе стелали пъачилйво и полудине заксизалйвыь шалл ма послетвее денчеъное матавие куъса гоня коведво я видеро ве ошеэащ


  0%|          | 0/1000000 [00:00<?, ?it/s]

Epoch: 9, Log-likelihood: -1210.2653009748797

если чы читине воъзалйвыь или подни воъзалйвыь нексн у жноро соошэевия коноъыь лерко пъодинанй скоъее чсеро чы чсе стелали пъачилйво и полудине заксизалйвыь шалл ма послетвее денчеъное матавие куъса гоня коведво я видеро ве ошеэащ


<i><strong>
Текст прочитать можно, но есть неточности
</strong></i>